# Final Project

#### The Plan
You're planning an evening out on the town with friends. You would rather your activities be nearby each other so you only have to park/take the bus/subway once, then walk to each location. You need to find a cluster of bars/restaurants/activities to do in one area with a set distance. You could use Google and attempt to make your own path, but wouldn't it be easier if you could just pick a starting point, say the activities you'd like to do, and a max walking distance, then get a map with all the points on it?

We will use the Foursquare data to give a demonstration of this for the Tulsa, Oklahoma area (limiting the data size that we pull for this example). We will use the venue category to determine the activities and the latitude and longitude to determine distance.

In [113]:
#User Input: Where do you want to start?
Starting_Point = 'Duet Jazz' #Name of Venue starting point.
Max_Distance_Travel = 0.5 #Distance in km.

In [114]:
#Import the needed packages:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0 --yes
import pandas as pd
import io
import requests
from bs4 import BeautifulSoup
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import numpy as np
import geopy.distance

In [115]:
#Latitude and Longitude of city for pulling in Foursquare data.
latitude = 36.1540 
longitude = -95.9928

In [116]:
#Import Foursquare data for desired city.
CLIENT_ID = 'OP2B2VRR31XUUOI4UH4D0UUPRFOKYMX41ZSOT3IEARQGNORB' # your Foursquare ID
CLIENT_SECRET = 'PGCLF1ZWRFFO4Y40SECHTSXS4Z4DGY3PI3YPINPS3FAAV2ZC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius=500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

Your credentails:
CLIENT_ID: OP2B2VRR31XUUOI4UH4D0UUPRFOKYMX41ZSOT3IEARQGNORB
CLIENT_SECRET:PGCLF1ZWRFFO4Y40SECHTSXS4Z4DGY3PI3YPINPS3FAAV2ZC


'https://api.foursquare.com/v2/venues/search?client_id=OP2B2VRR31XUUOI4UH4D0UUPRFOKYMX41ZSOT3IEARQGNORB&client_secret=PGCLF1ZWRFFO4Y40SECHTSXS4Z4DGY3PI3YPINPS3FAAV2ZC&ll=36.154,-95.9928&v=20180604&radius=500&limit=100'

In [117]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea794fddd0f850022a8ed85'},
 'response': {'venues': [{'id': '4d3f1d3cd625224b861ba9fa',
    'name': 'Williams Tower 2',
    'location': {'address': '2 W 2nd St',
     'lat': 36.15442650792134,
     'lng': -95.99130394098663,
     'labeledLatLngs': [{'label': 'display',
       'lat': 36.15442650792134,
       'lng': -95.99130394098663}],
     'distance': 142,
     'postalCode': '74103',
     'cc': 'US',
     'city': 'Tulsa',
     'state': 'OK',
     'country': 'United States',
     'formattedAddress': ['2 W 2nd St', 'Tulsa, OK 74103', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d124941735',
      'name': 'Office',
      'pluralName': 'Offices',
      'shortName': 'Office',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588041418',
    'hasPerk': False},
   {'id': '4b5bc3e5f964a520da1429e3',
    'name': 'Hyatt Regency Tulsa',


In [118]:
venues=results['response']['venues']
df2=json_normalize(venues)
df2

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4d3f1d3cd625224b861ba9fa,2 W 2nd St,US,Tulsa,United States,NaN,142,"[2 W 2nd St, Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15442650792134...",36.154427,-95.991304,NaN,74103,OK,Williams Tower 2,v-1588041418,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4b5bc3e5f964a520da1429e3,100 E Second St,US,Tulsa,United States,NaN,178,"[100 E Second St, Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15425202017010...",36.154252,-95.990842,NaN,74103,OK,Hyatt Regency Tulsa,v-1588041418,NaN
2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,5137b794e4b0db6be21be233,NaN,US,Tulsa,United States,NaN,212,"[Tulsa, OK, United States]","[{'label': 'display', 'lat': 36.15397566039657...",36.153976,-95.990433,NaN,NaN,OK,Hyatt Stay Fit,v-1588041418,NaN
3,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",False,5a5a218d23a2e63d53a10b4d,NaN,US,Tulsa,United States,NaN,2,"[Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.153988, 'lng':...",36.153988,-95.992775,NaN,74103,OK,Panera Bread,v-1588041418,NaN
4,"[{'id': '4bf58dd8d48988d174941735', 'name': 'C...",False,4f318c56e4b0acb1ebfb15ad,NaN,US,Tulsa,United States,NaN,47,"[Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15386659928187...",36.153867,-95.992297,NaN,74103,OK,State of Panic,v-1588041418,NaN
5,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,543d5920498e062420441f11,202 S Cheyenne Ave,US,Tulsa,United States,at W 2nd St,158,"[202 S Cheyenne Ave (at W 2nd St), Tulsa, OK 7...","[{'label': 'display', 'lat': 36.15335554056855...",36.153356,-95.994375,Downtown,74103,OK,All About Cha,v-1588041418,NaN
6,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",False,4f593e4fe4b0a5edcfbcb79e,112 W. 3rd Street,US,Tulsa,United States,NaN,158,"[112 W. 3rd Street, Tulsa, OK 74103, United St...","[{'label': 'display', 'lat': 36.15260533918836...",36.152605,-95.992443,NaN,74103,OK,SP+ Parking,v-1588041418,NaN
7,"[{'id': '4bf58dd8d48988d1f4941735', 'name': 'D...",False,4db892f0a86ee4fa7d261ae7,NaN,US,NaN,United States,NaN,70,"[Oklahoma, United States]","[{'label': 'display', 'lat': 36.15421270737053...",36.154213,-95.992060,NaN,NaN,Oklahoma,Matrix AE,v-1588041418,NaN
8,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4b98fc9cf964a5207d5a35e3,NaN,US,Tulsa,United States,NaN,161,"[Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15339400966457...",36.153394,-95.994436,NaN,74103,OK,Tulsa Regional Chamber,v-1588041418,NaN
9,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,4e4ddce1bd4101d0d79d02b5,200 Civic Ctr,US,Tulsa,United States,NaN,155,"[200 Civic Ctr, Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15408659463905...",36.154087,-95.991068,NaN,74103,OK,SUBWAY,v-1588041418,NaN


In [119]:
filtered_columns=['name','categories']+[col for col in df2.columns if col.startswith('location.')]+['id']
df_clean=df2.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    
df_clean['categories']=df_clean.apply(get_category_type, axis=1)
df_clean.columns=[column.split('.')[-1] for column in df_clean.columns]
df_clean.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Williams Tower 2,Office,2 W 2nd St,US,Tulsa,United States,NaN,142,"[2 W 2nd St, Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15442650792134...",36.154427,-95.991304,NaN,74103,OK,4d3f1d3cd625224b861ba9fa
1,Hyatt Regency Tulsa,Hotel,100 E Second St,US,Tulsa,United States,NaN,178,"[100 E Second St, Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15425202017010...",36.154252,-95.990842,NaN,74103,OK,4b5bc3e5f964a520da1429e3
2,Hyatt Stay Fit,Gym,NaN,US,Tulsa,United States,NaN,212,"[Tulsa, OK, United States]","[{'label': 'display', 'lat': 36.15397566039657...",36.153976,-95.990433,NaN,NaN,OK,5137b794e4b0db6be21be233
3,Panera Bread,Bakery,NaN,US,Tulsa,United States,NaN,2,"[Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.153988, 'lng':...",36.153988,-95.992775,NaN,74103,OK,5a5a218d23a2e63d53a10b4d
4,State of Panic,Coworking Space,NaN,US,Tulsa,United States,NaN,47,"[Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15386659928187...",36.153867,-95.992297,NaN,74103,OK,4f318c56e4b0acb1ebfb15ad


In [120]:
#Now that the data is cleaned, let's look at the categories for the city, so we can narrow down to the ones that we think could be considered a fun activity.
df_clean.categories.unique()

array(['Office', 'Hotel', 'Gym', 'Bakery', 'Coworking Space',
       'Coffee Shop', 'Parking', 'Design Studio', 'Sandwich Place',
       'Courthouse', 'Sushi Restaurant', 'Public Art',
       'Italian Restaurant', 'Bus Line', 'Music Venue',
       'College Arts Building', 'Building', 'Corporate Cafeteria',
       'Tapas Restaurant', 'Warehouse Store', 'Movie Theater',
       'Fast Food Restaurant', None, 'Food Court', 'Convention Center',
       'Hot Dog Joint', 'Restaurant', 'Lawyer', 'Event Space',
       "Doctor's Office", 'Nursery School', 'Tech Startup',
       'Professional & Other Places', 'Flower Shop', 'Bank'], dtype=object)

In [121]:
#Create a dataframe of just the locations whose categories indicate that they might be fun or food.
places = 'Bakery','Coffee Shop','Italian Restaurant','Sandwich Place','Public Art','Sushi Restaurant','Music Venue','Tapas Restaurant','Movie Theater','Fast Food Restaurant','Food Court','Hot Dog Joint','Restaurant','Event Space'
df_cat = df_clean.loc[df_clean['categories'].isin(places)]
df_cat.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
3,Panera Bread,Bakery,NaN,US,Tulsa,United States,NaN,2,"[Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.153988, 'lng':...",36.153988,-95.992775,NaN,74103,OK,5a5a218d23a2e63d53a10b4d
5,All About Cha,Coffee Shop,202 S Cheyenne Ave,US,Tulsa,United States,at W 2nd St,158,"[202 S Cheyenne Ave (at W 2nd St), Tulsa, OK 7...","[{'label': 'display', 'lat': 36.15335554056855...",36.153356,-95.994375,Downtown,74103,OK,543d5920498e062420441f11
9,SUBWAY,Sandwich Place,200 Civic Ctr,US,Tulsa,United States,NaN,155,"[200 Civic Ctr, Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15408659463905...",36.154087,-95.991068,NaN,74103,OK,4e4ddce1bd4101d0d79d02b5
12,The Sushi Place,Sushi Restaurant,115 W 3rd St Ste 100,US,Tulsa,United States,Cheyenne Ave.,135,"[115 W 3rd St Ste 100 (Cheyenne Ave.), Tulsa, ...","[{'label': 'display', 'lat': 36.1527885775784,...",36.152789,-95.992973,NaN,74103,OK,4ded78f952b13dda25e2a039
14,Riding Tulsa Trolley,Public Art,NaN,US,Tulsa,United States,NaN,93,"[Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15444928780553...",36.154449,-95.993682,NaN,74103,OK,50c2a3cce4b0e77d2954b620


In [122]:
#List of places avaliable.
df_clean.name.unique()

array(['Williams Tower 2', 'Hyatt Regency Tulsa', 'Hyatt Stay Fit',
       'Panera Bread', 'State of Panic', 'All About Cha', 'SP+ Parking',
       'Matrix AE', 'Tulsa Regional Chamber', 'SUBWAY',
       'Federal Building', 'The Sushi Place', 'Williams Center Tower 1',
       'Riding Tulsa Trolley', 'Central Parking', 'The Forge',
       'Ti Amo Ristorante Italiano', 'Bus 4 Yo',
       '100 West Parking Garage', "Uncle Bentley's Pub",
       'AR Workshop Tulsa', 'Zayo Bandwidth', 'Cimarex Energy',
       'Williams Cafeteria', 'Duet Jazz', 'INCOG',
       'Metropolitan Environmental Trust', "Sam's Club",
       'Samson Investment Company', 'Red Box', "Uncle Bently's", 'Kfc',
       'Centric', "Jimmey's", 'PRSA Southwest Regional Conference',
       'Hyatt Valet', 'Cains Ballroom - Tulsa', 'Downtown Coney',
       'Guckenheimer',
       'Lorton Performing Arts Center, University of Tulsa',
       'Michael French Law Office', 'JD Young', 'Morgan Studio',
       'national dialogue', 'Nicks

In [123]:
#Find the latitude and longitude for the starting place that the user requested.
place_lat = df_cat.loc[df_cat['name'] == Starting_Point, 'lat'].iloc[0]
place_lon = df_cat.loc[df_cat['name'] == Starting_Point, 'lng'].iloc[0]

print('Starting Latitude is:')
print(place_lat)
print('Starting Longitude is:')
print(place_lon)

Starting Latitude is:
36.15398
Starting Longitude is:
-95.99277


In [124]:
#Create a new column in our database that shows the distance of each location from the desired starting point.
for label, row in df_cat.iterrows():
    df_cat.loc[label,'dist_diff'] = geopy.distance.geodesic((place_lat, place_lon), (df_cat.loc[label,'lat'],df_cat.loc[label,'lng'])).km
print(df_cat)

                          name            categories               address  \
3                 Panera Bread                Bakery                   NaN   
5                All About Cha           Coffee Shop    202 S Cheyenne Ave   
9                       SUBWAY        Sandwich Place         200 Civic Ctr   
12             The Sushi Place      Sushi Restaurant  115 W 3rd St Ste 100   
14        Riding Tulsa Trolley            Public Art                   NaN   
17  Ti Amo Ristorante Italiano    Italian Restaurant    219 S Cheyenne Ave   
21         Uncle Bentley's Pub           Music Venue                   NaN   
27                   Duet Jazz      Tapas Restaurant                   NaN   
33                     Red Box         Movie Theater                   NaN   
34              Uncle Bently's           Music Venue                   NaN   
35                         Kfc  Fast Food Restaurant               admiral   
37                    Jimmey's            Food Court            

In [125]:
#Create a new dataframe that included only the locations within the desired radius of the starting location.
df_fin = df_cat.loc[df_cat['dist_diff'] <= Max_Distance_Travel]
df_fin

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,dist_diff
3,Panera Bread,Bakery,NaN,US,Tulsa,United States,NaN,2,"[Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.153988, 'lng':...",36.153988,-95.992775,NaN,74103,OK,5a5a218d23a2e63d53a10b4d,0.000995
5,All About Cha,Coffee Shop,202 S Cheyenne Ave,US,Tulsa,United States,at W 2nd St,158,"[202 S Cheyenne Ave (at W 2nd St), Tulsa, OK 7...","[{'label': 'display', 'lat': 36.15335554056855...",36.153356,-95.994375,Downtown,74103,OK,543d5920498e062420441f11,0.160189
9,SUBWAY,Sandwich Place,200 Civic Ctr,US,Tulsa,United States,NaN,155,"[200 Civic Ctr, Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15408659463905...",36.154087,-95.991068,NaN,74103,OK,4e4ddce1bd4101d0d79d02b5,0.153586
12,The Sushi Place,Sushi Restaurant,115 W 3rd St Ste 100,US,Tulsa,United States,Cheyenne Ave.,135,"[115 W 3rd St Ste 100 (Cheyenne Ave.), Tulsa, ...","[{'label': 'display', 'lat': 36.1527885775784,...",36.152789,-95.992973,NaN,74103,OK,4ded78f952b13dda25e2a039,0.133456
14,Riding Tulsa Trolley,Public Art,NaN,US,Tulsa,United States,NaN,93,"[Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15444928780553...",36.154449,-95.993682,NaN,74103,OK,50c2a3cce4b0e77d2954b620,0.097163
17,Ti Amo Ristorante Italiano,Italian Restaurant,219 S Cheyenne Ave,US,Tulsa,United States,W 3rd St,159,"[219 S Cheyenne Ave (W 3rd St), Tulsa, OK 7410...","[{'label': 'display', 'lat': 36.15271398211045...",36.152714,-95.993574,NaN,74103,OK,4b672f95f964a5201c402be3,0.158015
21,Uncle Bentley's Pub,Music Venue,NaN,US,Tulsa,United States,NaN,3,"[Tulsa, OK, United States]","[{'label': 'display', 'lat': 36.15398, 'lng': ...",36.153980,-95.992770,NaN,NaN,OK,575d11ad498ea666957d3993,0.000000
27,Duet Jazz,Tapas Restaurant,NaN,US,Tulsa,United States,NaN,3,"[Tulsa, OK, United States]","[{'label': 'display', 'lat': 36.15398, 'lng': ...",36.153980,-95.992770,NaN,NaN,OK,5b5830c22be425002c44ec74,0.000000
33,Red Box,Movie Theater,NaN,US,Tulsa,United States,NaN,82,"[Tulsa, OK 74103, United States]","[{'label': 'display', 'lat': 36.15470104981796...",36.154701,-95.992520,NaN,74103,OK,4f1b7210e4b0a627191c5ddb,0.083123
34,Uncle Bently's,Music Venue,NaN,US,Tulsa,United States,NaN,3,"[Tulsa, OK, United States]","[{'label': 'display', 'lat': 36.15398, 'lng': ...",36.153980,-95.992770,NaN,NaN,OK,57da50ec498ea76140fb9d04,0.000000


In [133]:
#Create a map of the final dataframe which shows the location of each point of potential fun.
funmap = folium.Map(location=[df_fin['lat'],df_fin['lng']], zoom_start=50)

for lat, lon, name in zip(df_fin['lat'], df_fin['lng'], df_fin['name']):
    folium.CircleMarker(
         [lat, lon],
         radius=5,
         popup=folium.Popup(str(name), parse_html=True),
         color='purple',
         fill=True,
         fill_color='blue',
         fill_opacity=0.7).add_to(funmap)
       
funmap

TypeError: cannot convert the series to <class 'float'>